Download packages and install libraries

In [ ]:
!pip install transformers
!pip install decord
!pip install moviepy
!pip install python-magic
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow transformers

from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification, pipeline
from moviepy.editor import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import magic
import os
from IPython.display import HTML
from base64 import b64encode

Set up pipeline to get Inference

In [ ]:
pipe = pipeline("video-classification", model="anirudhmu/videomae-base-finetuned-soccer-action-recognitionx4")

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/345M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/471 [00:00<?, ?B/s]

Function 'predict_video' to

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os
import magic
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip

# Initialize a dictionary to hold the count of occurrences for each event
event_count = {}
#setpiece_times = []
def predict_video(file_path):
    # Check the file type
    mime = magic.Magic(mime=True)
    file_type = mime.from_file(file_path)

    if not file_type.startswith('video/'):
        print("Invalid file type. Please upload a valid video.")
        return

    # Load the video file
    video = VideoFileClip(file_path)
    duration = video.duration
    clip_duration = 10  # duration of clips in seconds
    setpiece_cooldown = 20  # cooldown time in seconds after a event detection
    setpiece_last_detected = -setpiece_cooldown  # initialize to negative cooldown to allow detection from start
    confidence_threshold = 0.995  # confidence threshold
    fps = 25

    # Split the video into clips and perform inference
    for start in range(0, int(duration), clip_duration):
        end = min(start + clip_duration, duration)
        clip_path = f"temp_clip_{start}_{end}.mp4"
        ffmpeg_extract_subclip(file_path, start, end, targetname=clip_path)

        print(f"Processing clip: {clip_path}")
        results = pipe(clip_path)

        # Sort results by score in descending order to get the highest score first
        sorted_results = sorted(results, key=lambda result: result["score"], reverse=True)
        highest_score_result = sorted_results[0]

        # Check if the highest score label is 'Setpiece' and above the confidence threshold
        if (highest_score_result["label"] == 'Setpiece' or highest_score_result["label"] == 'Penalty') and highest_score_result["score"] > confidence_threshold:
            # If we are past the cooldown period, or if the label is not 'Setpiece' or 'Penalty'
            if start - setpiece_last_detected >= setpiece_cooldown:
                # Update the last detected time and the event count
                event_name = highest_score_result["label"]
                setpiece_last_detected = start
                event_count['Setpiece'] = event_count.get('Setpiece', 0) + 1
                print(f"Detected {event_name} with confidence {highest_score_result['score']}")

                # Display the video clip
                mp4 = open(clip_path,'rb').read()
                data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
                display(HTML(f'''
                <video width=400 controls>
                       <source src="{data_url}" type="video/mp4">
                </video>
                '''))
            else:
                print(f"Ignoring {event_name} detection due to cooldown.")
        else:
            if highest_score_result["score"] > confidence_threshold:
                # Update the event count for labels other than 'Setpiece' if confidence is high enough
                event_count[highest_score_result["label"]] = event_count.get(highest_score_result["label"], 0) + 1
                print(f"Detected label: {highest_score_result['label']} with confidence {highest_score_result['score']}")



        # Remove the temporary video clip file
        os.remove(clip_path)

    # After all clips are processed, print the event count
    print("Event count:", event_count)



In [ ]:
#Predict on 1st half
predict_video('overview_live.mp4')

In [ ]:
#Predict on 2nd half
predict_video('overview_live2.mp4')